In [1]:
import os
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from skimage import io, transform
from torchvision import transforms, utils
import torch
import numpy as np
import nibabel as nib
from random import randint
from PIL import Image
import torch.optim as optim
import time
import QuickNAT as QN
import torch.nn as nn
from my_Dataset import TrainDataset

In [2]:
gpu_id = 0
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

device = torch.device('cuda')
print(device)

cuda


In [3]:
sub_idx = 0
slice_idx = 3
T1a_dir = '/home/xiaoyu/MRIdata_group/T1w/axial/sub{}/slice{}'.format(sub_idx,slice_idx)
parc1a_dir = '/home/xiaoyu/MRIdata_group/parc_1/axial/sub{}/slice{}'.format(sub_idx,slice_idx)
total_data_0 = TrainDataset(T1a_dir=T1a_dir, parc1a_dir = parc1a_dir)

In [4]:
for sub_idx in range(1,330):
    T1a_dir = '/home/xiaoyu/MRIdata_group/T1w/axial/sub{}/slice{}'.format(sub_idx,slice_idx)
    parc1a_dir = '/home/xiaoyu/MRIdata_group/parc_1/axial/sub{}/slice{}'.format(sub_idx,slice_idx)
    train_data = TrainDataset(T1a_dir=T1a_dir, parc1a_dir = parc1a_dir)
    total_data_0 = total_data_0 + train_data
print(len(total_data_0))

3300


In [5]:
dataloader_0 = DataLoader(total_data_0, batch_size = 10, shuffle = True, num_workers = 4)
print(len(dataloader_0))

330


In [6]:
start=time.time()
model = QN.QuickNAT(1,64,178)
nb_param=0
for param in model.parameters():
    nb_param+=np.prod(list(param.data.size()))
print(nb_param)
model = model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters() ,lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

3303618


In [7]:
for epoch in range(0,120):
    running_loss = 0
    num_batches = 0
    scheduler.step()
    for i_batch, sample_batched in enumerate(dataloader_0):
    
        optimizer.zero_grad()
        
        #get the inputs
        inputs, labels = sample_batched['T1a'], sample_batched['parc1a']
        

        inputs = inputs.to(device)
        labels = labels.to(device)

        inputs.requires_grad_()
        
        #forward + backward +optimize
        scores = model(inputs)

          
        # Define the loss
        loss = criterion(scores, labels.long()) 
        loss.backward()
        optimizer.step()
        
        # compute and accumulate stats
        running_loss += loss.detach().item()
       
        num_batches+=1 
        
        
    # AVERAGE STATS THEN DISPLAY    
    total_loss = running_loss/num_batches
   
    elapsed = (time.time()-start)/60
        
    print('epoch=',epoch, '\t time=', elapsed,'min', '\t loss=', total_loss ) 
 
print('Finish Training')

epoch= 0 	 time= 1.12700643936793 min 	 loss= 0.9667042183153557
epoch= 1 	 time= 2.2239407618840534 min 	 loss= 0.716198267178102
epoch= 2 	 time= 3.315772557258606 min 	 loss= 0.6992378413677216
epoch= 3 	 time= 4.404611965020497 min 	 loss= 0.6855370832212043
epoch= 4 	 time= 5.496378405888875 min 	 loss= 0.6685499420671752
epoch= 5 	 time= 6.5886075615882875 min 	 loss= 0.6524926628127242
epoch= 6 	 time= 7.694467310110728 min 	 loss= 0.6358752256089991
epoch= 7 	 time= 8.786052135626475 min 	 loss= 0.6252101933414286
epoch= 8 	 time= 9.879029393196106 min 	 loss= 0.6164258637211539
epoch= 9 	 time= 10.980497260888418 min 	 loss= 0.6098581777377562
epoch= 10 	 time= 12.083403209845224 min 	 loss= 0.604097845879468


KeyboardInterrupt: 